In [1]:
from skfem import *
import numpy as np
from utils import solver_iter_krylov, solver_iter_pyamg, solver_iter_mgcg
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time

# aaa

In [2]:
for ii in range(1):
    pi = np.pi
    sin = np.sin
    cos = np.cos
    exp = np.exp

    # parameters

    tol = 1e-8
    intorder = 5
    refine_time = 5
    epsilon_range = 1
    element_type = 'P1'
    sigma = 5
    penalty = False
    example = 'ex1'

    # end of parameters

    save_path = 'log/' + example + '_' + element_type + '_' + (
        'pen' if penalty else 'nopen') + '_' + '{}'.format(
            datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

    # # output to txt
    # class Logger(object):
    #     def __init__(self, filename=save_path+'.txt', stream=sys.stdout):
    # 	    self.terminal = stream
    # 	    self.log = open(filename, 'a')

    #     def write(self, message):
    # 	    self.terminal.write(message)
    # 	    self.log.write(message)

    #     def flush(self):
    # 	    pass

    # sys.stdout = Logger(save_path+'.txt', sys.stdout)

    # print parameters

    print('=======Arguments=======')
    print('example:\t{}'.format(example))
    print('penalty:\t{}'.format(penalty))
    print('element_type:\t{}'.format(element_type))
    print('tol:\t{}'.format(tol))
    print('intorder:\t{}'.format(intorder))
    print('refine_time:\t{}'.format(refine_time))
    print('epsilon_range:\t{}'.format(epsilon_range))
    print('sigma:\t{}'.format(sigma))
    print('save_path:\t{}'.format(save_path))
    print('=======Results=======')

    # functions


    def easy_boundary_penalty(basis):
        '''
        Input basis
        ----------------
        Return D for boundary conditions
        '''

        dofs = basis.find_dofs({
            'left': m.facets_satisfying(lambda x: x[0] == 0),
            'right': m.facets_satisfying(lambda x: x[0] == 1),
            'top': m.facets_satisfying(lambda x: x[1] == 1),
            'buttom': m.facets_satisfying(lambda x: x[1] == 0)
        })

        D = np.concatenate((dofs['left'].nodal['u'], dofs['right'].nodal['u'],
                            dofs['top'].nodal['u'], dofs['buttom'].nodal['u']))
        return D


    def easy_boundary(basis):
        '''
        Input basis
        ----------------
        Return D for boundary conditions
        '''

        dofs = basis.find_dofs({
            'left': m.facets_satisfying(lambda x: x[0] == 0),
            'right': m.facets_satisfying(lambda x: x[0] == 1),
            'top': m.facets_satisfying(lambda x: x[1] == 1),
            'buttom': m.facets_satisfying(lambda x: x[1] == 0)
        })

        D = np.concatenate((dofs['left'].nodal['u'], dofs['right'].nodal['u'],
                            dofs['top'].nodal['u'], dofs['buttom'].nodal['u'],
                            dofs['left'].facet['u_n'], dofs['right'].facet['u_n'],
                            dofs['top'].facet['u_n'], dofs['buttom'].facet['u_n']))
        return D


    @BilinearForm
    def a_load(u, v, w):
        '''
        for $a_{h}$
        '''
        return ddot(dd(u), dd(v))


    @BilinearForm
    def b_load(u, v, w):
        '''
        for $b_{h}$
        '''
        return dot(grad(u), grad(v))


    @BilinearForm
    def wv_load(u, v, w):
        '''
        for $(\nabla \chi_{h}, \nabla_{h} v_{h})$
        '''
        return dot(grad(u), grad(v))


    @BilinearForm
    def penalty_1(u, v, w):
        return ddot(-dd(u), prod(w.n, w.n)) * dot(grad(v), w.n)


    @BilinearForm
    def penalty_2(u, v, w):
        return ddot(-dd(v), prod(w.n, w.n)) * dot(grad(u), w.n)


    @BilinearForm
    def penalty_3(u, v, w):
        return (sigma / w.h) * dot(grad(u), w.n) * dot(grad(v), w.n)


    @BilinearForm
    def laplace(u, v, w):
        '''
        for $(\nabla w_{h}, \nabla \chi_{h})$
        '''
        return dot(grad(u), grad(v))


    @Functional
    def L2uError(w):
        x, y = w.x
        return (w.w - exact_u(x, y))**2


    def get_DuError(basis, u):
        duh = basis.interpolate(u).grad
        x = basis.global_coordinates().value
        dx = basis.dx  # quadrature weights
        dux, duy = dexact_u(x[0], x[1])
        return np.sqrt(np.sum(((duh[0] - dux)**2 + (duh[1] - duy)**2) * dx))


    def get_D2uError(basis, u):
        dduh = basis.interpolate(u).hess
        x = basis.global_coordinates(
        ).value  # coordinates of quadrature points [x, y]
        dx = basis.dx  # quadrature weights
        duxx, duxy, duyx, duyy = ddexact(x[0], x[1])
        return np.sqrt(
            np.sum(((dduh[0][0] - duxx)**2 + (dduh[0][1] - duxy)**2 +
                    (dduh[1][1] - duyy)**2 + (dduh[1][0] - duyx)**2) * dx))


    def solve_problem1(m, element_type='P1'):
        '''
        old solver for problem 1, can't read f 
        '''
        global K1
        if element_type == 'P1':
            element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
        elif element_type == 'P2':
            element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
        else:
            raise Exception("The element not supported")

        basis = {
            variable: InteriorBasis(m, e, intorder=intorder)
            for variable, e in element.items()
        }  # intorder: integration order for quadrature

        K1 = asm(laplace, basis['w'])
        f1 = asm(f_load, basis['w'])

        # wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))
        wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()),
                   solver=solver_iter_krylov(Precondition=True, tol=tol))

        K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
        f2 = asm(wv_load, basis['w'], basis['u']) * wh
        uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])),
                    solver=solver_iter_krylov(Precondition=True, tol=tol))  # cg
        return uh0, basis


    def solve_problem2(m, element_type='P1'):
        '''
        old solver
        '''
        if element_type == 'P1':
            element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
        elif element_type == 'P2':
            element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
        else:
            raise Exception("The element not supported")

        basis = {
            variable: InteriorBasis(m, e, intorder=intorder)
            for variable, e in element.items()
        }

        K1 = asm(laplace, basis['w'])
        f1 = asm(f_load, basis['w'])

        wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()),
                   solver=solver_iter_krylov(Precondition=True, tol=tol))

        fbasis = FacetBasis(m, element['u'])

        p1 = asm(penalty_1, fbasis)
        p2 = asm(penalty_2, fbasis)
        p3 = asm(penalty_3, fbasis)
        P = p1 + p2 + p3

        K2 = epsilon**2 * asm(a_load, basis['u']) + \
            epsilon**2 * P + asm(b_load, basis['u'])
        f2 = asm(wv_load, basis['w'], basis['u']) * wh
        uh0 = solve(*condense(K2, f2, D=easy_boundary_penalty(basis['u'])),
                    solver=solver_iter_krylov(Precondition=True, tol=tol))
        return uh0, basis


    if example == 'ex1':

        @LinearForm
        def f_load(v, w):
            '''
            for $(f, x_{h})$
            '''
            pix = pi * w.x[0]
            piy = pi * w.x[1]
            lu = 2 * (pi)**2 * (cos(2 * pix) * ((sin(piy))**2) + cos(2 * piy) *
                                ((sin(pix))**2))
            llu = -8 * (pi)**4 * (cos(2 * pix) * sin(piy)**2 + cos(2 * piy) *
                                  sin(pix)**2 - cos(2 * pix) * cos(2 * piy))
            return (epsilon**2 * llu - lu) * v

        def exact_u(x, y):
            return (sin(pi * x) * sin(pi * y))**2

        def dexact_u(x, y):
            dux = 2 * pi * cos(pi * x) * sin(pi * x) * sin(pi * y)**2
            duy = 2 * pi * cos(pi * y) * sin(pi * x)**2 * sin(pi * y)
            return dux, duy

        def ddexact(x, y):
            duxx = 2 * pi**2 * cos(pi * x)**2 * sin(pi * y)**2 - 2 * pi**2 * sin(
                pi * x)**2 * sin(pi * y)**2
            duxy = 2 * pi * cos(pi * x) * sin(pi * x) * 2 * pi * cos(pi * y) * sin(
                pi * y)
            duyx = duxy
            duyy = 2 * pi**2 * cos(pi * y)**2 * sin(pi * x)**2 - 2 * pi**2 * sin(
                pi * y)**2 * sin(pi * x)**2
            return duxx, duxy, duyx, duyy

    elif example == 'ex2':

        @LinearForm
        def f_load(v, w):
            '''
            for $(f, x_{h})$
            '''
            x = w.x[0]
            y = w.x[1]
            return ((sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) / (2 * (exp(-1 / ep) - 1))) *
                    (12 * y + ep *
                     ((exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                       (exp(-1 / ep) + 2 * ep *
                                        (exp(-1 / ep) - 1) + 1))) / ep**2 +
                      (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                            (exp(-1 / ep) + 2 * ep *
                                             (exp(-1 / ep) - 1) + 1))) / ep**2)) -
                    ((pi**2 * sin(pi * x)) / 2 +
                     (ep * pi * (exp(-x / ep) / ep**2 + exp(
                         (x - 1) / ep) / ep**2)) / (2 * (exp(-1 / ep) - 1))) *
                    (ep * (exp(
                        (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                         (exp(-1 / ep) + 2 * ep *
                                          (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                           (3 / (exp(-1 / ep) - 1) + 1 /
                            (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1)) -
                           (3 * exp(-1 / ep) + 3) / (exp(-1 / ep) - 1) -
                           ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                           (exp(-1 / ep) + 2 * ep *
                            (exp(-1 / ep) - 1) + 1)) + 2 * y *
                     (y**2 - 1)) - ep**2 *
                    (((pi**4 * sin(pi * x)) / 2 -
                      (ep * pi * (exp(-x / ep) / ep**4 + exp(
                          (x - 1) / ep) / ep**4)) / (2 * (exp(-1 / ep) - 1))) *
                     (ep * (exp((y - 1) / ep) *
                            (3 / (exp(-1 / ep) - 1) - 1 /
                             (exp(-1 / ep) + 2 * ep *
                              (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                            (3 / (exp(-1 / ep) - 1) + 1 /
                             (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1)) -
                            (3 * exp(-1 / ep) + 3) / (exp(-1 / ep) - 1) -
                            ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                            (exp(-1 / ep) + 2 * ep *
                             (exp(-1 / ep) - 1) + 1)) + 2 * y * (y**2 - 1)) - 2 *
                     (12 * y + ep *
                      ((exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                        (exp(-1 / ep) + 2 * ep *
                                         (exp(-1 / ep) - 1) + 1))) / ep**2 +
                       (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                             (exp(-1 / ep) + 2 * ep *
                                              (exp(-1 / ep) - 1) + 1))) / ep**2)) *
                     ((pi**2 * sin(pi * x)) / 2 +
                      (ep * pi * (exp(-x / ep) / ep**2 + exp(
                          (x - 1) / ep) / ep**2)) / (2 *
                                                     (exp(-1 / ep) - 1))) + ep *
                     (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                         (x - 1) / ep) - exp(-1 / ep) - 1)) /
                      (2 * (exp(-1 / ep) - 1))) *
                     ((exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                       (exp(-1 / ep) + 2 * ep *
                                        (exp(-1 / ep) - 1) + 1))) / ep**4 +
                      (exp((y - 1) / ep) *
                       (3 / (exp(-1 / ep) - 1) - 1 /
                        (exp(-1 / ep) + 2 * ep *
                         (exp(-1 / ep) - 1) + 1))) / ep**4))) * v

        def exact_u(x, y):
            return -(sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) / (2 * (exp(-1 / ep) - 1))) * (
                    ep * (exp(
                        (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                         (exp(-1 / ep) + 2 * ep *
                                          (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                          (3 / (exp(-1 / ep) - 1) + 1 /
                           (exp(-1 / ep) + 2 * ep *
                            (exp(-1 / ep) - 1) + 1)) - (3 * exp(-1 / ep) + 3) /
                          (exp(-1 / ep) - 1) - ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                          (exp(-1 / ep) + 2 * ep *
                           (exp(-1 / ep) - 1) + 1)) + 2 * y * (y**2 - 1))

        def dexact_u(x, y):
            dux = -(
                (pi * cos(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep - exp(
                    (x - 1) / ep) / ep)) /
                (2 *
                 (exp(-1 / ep) - 1))) * (ep * (exp(
                     (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                      (exp(-1 / ep) + 2 * ep *
                                       (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                               (3 / (exp(-1 / ep) - 1) + 1 /
                                                (exp(-1 / ep) + 2 * ep *
                                                 (exp(-1 / ep) - 1) + 1)) -
                                               (3 * exp(-1 / ep) + 3) /
                                               (exp(-1 / ep) - 1) -
                                               ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                                               (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                         (y**2 - 1))
            duy = (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) /
                   (2 * (exp(-1 / ep) - 1))) * (ep * (
                       (2 * (exp(-1 / ep) - 1)) / (exp(-1 / ep) + 2 * ep *
                                                   (exp(-1 / ep) - 1) + 1) +
                       (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                        (exp(-1 / ep) + 2 * ep *
                                         (exp(-1 / ep) - 1) + 1))) / ep -
                       (exp((y - 1) / ep) *
                        (3 / (exp(-1 / ep) - 1) - 1 /
                         (exp(-1 / ep) + 2 * ep *
                          (exp(-1 / ep) - 1) + 1))) / ep) - 6 * y**2 + 2)
            return dux, duy

        def ddexact(x, y):
            duxx = (
                (pi**2 * sin(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep**2 + exp(
                    (x - 1) / ep) / ep**2)) /
                (2 *
                 (exp(-1 / ep) - 1))) * (ep * (exp(
                     (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                      (exp(-1 / ep) + 2 * ep *
                                       (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                               (3 / (exp(-1 / ep) - 1) + 1 /
                                                (exp(-1 / ep) + 2 * ep *
                                                 (exp(-1 / ep) - 1) + 1)) -
                                               (3 * exp(-1 / ep) + 3) /
                                               (exp(-1 / ep) - 1) -
                                               ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                                               (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                         (y**2 - 1))
            duxy = ((pi * cos(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep - exp(
                (x - 1) / ep) / ep)) / (2 * (exp(-1 / ep) - 1))) * (ep * (
                    (2 * (exp(-1 / ep) - 1)) / (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1) +
                    (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                     (exp(-1 / ep) + 2 * ep *
                                      (exp(-1 / ep) - 1) + 1))) / ep -
                    (exp((y - 1) / ep) *
                     (3 / (exp(-1 / ep) - 1) - 1 /
                      (exp(-1 / ep) + 2 * ep *
                       (exp(-1 / ep) - 1) + 1))) / ep) - 6 * y**2 + 2)
            duyx = duxy
            duyy = -(sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) /
                     (2 * (exp(-1 / ep) - 1))) * (12 * y + ep * (
                         (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                          (exp(-1 / ep) + 2 * ep *
                                           (exp(-1 / ep) - 1) + 1))) / ep**2 +
                         (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                               (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1))) / ep**2))
            return duxx, duxy, duyx, duyy

    elif example == 'ex3':

        @LinearForm
        def f_load(v, w):
            pix = pi * w.x[0]
            piy = pi * w.x[1]
            return (2 * pi**2 * sin(pix) * sin(piy)) * v

        def exact_u(x, y):
            return sin(pi * x) * sin(pi * y)

        def dexact_u(x, y):
            dux = pi * cos(pi * x) * sin(pi * y)
            duy = pi * cos(pi * y) * sin(pi * x)
            return dux, duy

        def ddexact(x, y):
            duxx = -pi**2 * sin(pi * x) * sin(pi * y)
            duxy = pi * cos(pi * x) * pi * cos(pi * y)
            duyx = duxy
            duyy = -pi**2 * sin(pi * y) * sin(pi * x)
            return duxx, duxy, duyx, duyy

    else:
        raise Exception('Example not supported')

=======Arguments=======
example:	ex1
penalty:	False
element_type:	P1
tol:	1e-08
intorder:	5
refine_time:	5
epsilon_range:	1
sigma:	5
save_path:	log/ex1_P1_nopen_2020-10-25_15-11-59
=======Results=======


# bbb

In [3]:
def solve_problem1(m, element_type='P1'):
    '''
    old solver for problem 1, can't read f 
    '''
    global K1
    if element_type == 'P1':
        element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
    elif element_type == 'P2':
        element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
    else:
        raise Exception("The element not supported")

    basis = {
        variable: InteriorBasis(m, e, intorder=intorder)
        for variable, e in element.items()
    }  # intorder: integration order for quadrature

    K1 = asm(laplace, basis['w'])
    f1 = asm(f_load, basis['w'])

    # wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))
    wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_pyamg(tol=tol))

    K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
    f2 = asm(wv_load, basis['w'], basis['u']) * wh
    uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])),
                solver=solver_iter_krylov(Precondition=True, tol=tol))  # cg
    return uh0, basis

time_start = time.time()

df_list = []
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()

        if penalty:
            uh0, basis = solve_problem2(m, element_type)
        else:
            uh0, basis = solve_problem1(m, element_type)

        U = basis['u'].interpolate(uh0).value

        # compute errors

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    # store data
    data = np.array([L2s, H1s, H2s, epus])
    df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
    df_list.append(df)

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))

time_end = time.time()

epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.83  0.87  0.71  0.70
2^-3  2.19  1.76  1.02  0.98
2^-4  2.16  1.93  1.05  1.02
2^-5  2.06  1.98  1.02  1.01


In [4]:
tol = 1e-10

In [11]:
m=MeshTri()
m.refine(8)
print((2**9)**2)

262144


In [12]:
if element_type == 'P1':
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
elif element_type == 'P2':
    element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
else:
    raise Exception("The element not supported")

In [13]:
%%time
basis = {
    variable: InteriorBasis(m, e, intorder=intorder)
    for variable, e in element.items()
}  # intorder: integration order for quadrature

Wall time: 12.6 s


In [14]:
epsilon = 1
ep = epsilon
tol = 1e-5

In [15]:
%%time
K1 = asm(laplace, basis['w'])
f1 = asm(f_load, basis['w'])

Wall time: 887 ms


# First part

- MGCG

In [17]:
%%time
# wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))
# wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_mgcg(tol=tol, verbose=True))
wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_mgcg(tol=tol))

Wall time: 168 ms


- PCG

In [18]:
%%time
# wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol, verbose=True))
wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))


Wall time: 409 ms


- AMG

In [19]:
%%time
wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_pyamg(tol=tol))

Wall time: 218 ms


# Assemble

In [20]:
%%time
K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
f2 = asm(wv_load, basis['w'], basis['u']) * wh

Wall time: 2.94 s


- MGCG

In [21]:
%%time
uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_mgcg(tol=tol))

# uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_mgcg(tol=tol, verbose=True))

Wall time: 26.7 s


- PCG

In [22]:
%%time
# uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_krylov(Precondition=True, tol=tol, verbose=True))  
uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_krylov(Precondition=True, tol=tol))  

Wall time: 1min 50s


- AMG

In [22]:
%%time 
uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_pyamg(tol=tol, verbose=True)) 

2.51068521502245
152.5106857505377
152.51068628390104
152.51068681512157
152.51068734420727
152.51068787116685
152.51068839600927
152.5106889187431
152.5106894393762
152.51068995791704
152.51069047437477
152.5106909887573
152.5106915010725
152.5106920113293
152.51069251953575
152.51069302570005
152.51069352983043
152.5106940319349
152.5106945320218
152.5106950300994
152.51069552617565
152.51069602025876
152.51069651235628
152.51069700247643
152.5106974906276
152.5106979768166
152.51069846105196
152.51069894334202
152.5106994236945
152.5106999021169
152.51070037861615
152.5107008532011
152.51070132587878
152.51070179665751
152.51070226554415
152.51070273254666
152.51070319767305
152.51070366093026
152.5107041223258
152.51070458186737
152.51070503956234
152.51070549541834
152.51070594944278
152.5107064016429
152.5107068520256
152.51070730059885
152.51070774736914
152.5107081923444
152.51070863553159
152.5107090769382
152.51070951657107
152.51070995443735
152.51071039054375
152.5107108248